# Day 22

又是比较坑的一天，主要障碍也还是理解题意，因为错误理解的意思，导致排查时间将近2个小时。其次就是第二部分时间复杂度很高，用Rust优化后仍需要将近2秒才出结果，用Python则需要几十秒，很痛苦。

## Part I

第一部分难度不高，使用collection.deque来表示两个玩家手中的牌，这样就可以方便的从前面出牌，赢了再从后面插入赢得的牌。这里使用字典表示玩家名称和手中的牌，实际上使用两个元素的数组也就够了，Rust当中就没有使用字典。

首先处理输入：

In [1]:
from collections import deque
from typing import Dict, Deque, Tuple

def pile_deck(data: str) -> Tuple[str, Deque[int]]:
    lines = [line for line in data.split('\n') if line]
    player = lines[0].rstrip(':')
    deck = deque()
    for card in lines[1:]:
        deck.append(int(card))
    return player, deck

def read_input(input_file: str) -> Dict[str, Deque[int]]:
    with open(input_file) as fn:
        content = fn.read()
    players = {}
    for player in content.split('\n\n'):
        p, d = pile_deck(player)
        players[p] = d
    return players

然后是第一部分的一轮游戏（game）的逻辑，只需要比较大小，然后将赢了的牌插入到队列后面：

In [2]:
def part1_next_round(players: Dict[str, Deque[int]]) -> str:
    player1 = players['Player 1']
    player2 = players['Player 2']
    card1 = player1.popleft()
    card2 = player2.popleft()
    if card1 > card2:
        player1.append(card1)
        player1.append(card2)
        return 'Player 1'
    player2.append(card2)
    player2.append(card1)
    return 'Player 2'

第一部分的逻辑，求加权总和：

In [3]:
def part1_solution(players: Dict[str, Deque[int]]) -> int:
    while all(len(deck) > 0 for deck in players.values()):
        winner = part1_next_round(players)
    return sum((i + 1) * card for i, card in enumerate(list(players[winner])[::-1]))

单元测试：

In [4]:
test_players = read_input('testcase1.txt')
assert(part1_solution(test_players) == 306)

第一部分结果：

In [5]:
players = read_input('input.txt')
part1_solution(players)

33400

## Part II

理解错了两个地方：

> Before either player deals a card, if there was a previous round in this game that had exactly the same cards in the same order in the same players' decks, the game instantly ends in a win for player 1.

我理解成任何一个玩家出现了曾经出现过的牌型，即判断为Player 1胜利。实际上是两个玩家都出现了曾经出现过的牌型，也就是关系是or，而不是and。

> not counting the 3 and 7 cards that were drawn.

漏看了这句，两个玩家本轮应该出的牌不计入sub game的牌之内。

这两个逻辑错误定位很困难，所以浪费了不少时间。

首先定义函数来实现第二轮游戏的逻辑：

In [6]:
def part2_game(players: Dict[str, Deque[int]]) -> str:
    # history用来记录本次game两个玩家手中持有的牌的全部历史记录
    history = []
    while True:
        winner, signal = part2_next_round(players, history)
        # 这里需要增加一个信号量，表示sub game是否已经结束
        if any(len(deck) == 0 for deck in players.values()) or signal == 'GAMEOVER':
            return winner

然后是第二部分每轮出牌和赢牌的逻辑：

In [7]:
from itertools import islice
from typing import List

def part2_next_round(players: Dict[str, Deque[int]], 
                     history: Dict[str, List[Deque[int]]]=None) -> Tuple[str, str]:
    player1 = players['Player 1']
    player2 = players['Player 2']
    # 如果存在历史记录
    if history and (player1, player2) in history:
        # 判断两个玩家的牌是否同时在历史记录中出现过，如果是，直接返回胜利方和GAMEOVER信号量
        return 'Player 1', 'GAMEOVER'
    # 将本轮两个玩家手中牌型记录在history中
    history.append((player1.copy(), player2.copy()))
    # 出牌
    card1 = player1.popleft()
    card2 = player2.popleft()
    # 如果出的牌满足进入子游戏sub game的条件
    if len(player1) >= card1 and len(player2) >= card2:
        sub_game_decks = {
            'Player 1': deque(islice(player1, 0, card1)),
            'Player 2': deque(islice(player2, 0, card2)),
        }
        winner = part2_game(sub_game_decks)
    else:
        winner = 'Player 1' if card1 > card2 else 'Player 2'
    if winner == 'Player 1':
        player1.append(card1)
        player1.append(card2)
    else:
        player2.append(card2)
        player2.append(card1)
#     print(history)
    return winner, 'ONEROUND'

第二部分实现逻辑：

In [8]:
def part2_solution(players: Dict[str, Deque[int]]) -> int:
    winner = part2_game(players)
    return sum((i + 1) * card for i, card in enumerate(list(players[winner])[::-1]))

单元测试：

In [9]:
test_players = read_input('testcase1.txt')
assert(part2_solution(test_players) == 291)

第二部分结果，需要等待几十秒，暂未想到好的优化方式：

In [10]:
players = read_input('input.txt')
part2_solution(players)

33745